Polymer Blend
===========

insert description

### Concepts Used
- blergh

### Tools Used
- blergh

References
-----------
1. Yethiraj, A.; Schweizer, K.S.; Integral equation theory of polymer blends: Numerical Investigation of molecular closure approximations, J. Chem. Phys., 1993, 98 (11) 
 
  

Global Imports and Setup
----------------------------

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh','matplotlib')

%load_ext autoreload
%autoreload 2

In [ ]:
import typyPRISM
import numpy as np

delta = 0.05#0.385
factor = 1.0/(1.0 - delta)

N = 8
sigma = l = 1.0
phi   = 0.5
eta   = 0.3
v1   = 4.0/3.0 * np.pi * (sigma/2.0)**(3.0)
rhoA = phi * eta/v1*factor
rhoB = (1-phi) * eta/v1*factor
eps_bend = 0.0001
kT = 1.0
print(rhoA,rhoB)
sys = typyPRISM.System(['A','B'],kT=kT)
sys.domain = typyPRISM.Domain(dr=0.05,length=2048)

sys.density['A'] = rhoA
sys.density['B'] = rhoB

sys.potential[sys.types,sys.types] = typyPRISM.potential.HardSphere(sigma=sigma)
sys.closure[sys.types,sys.types] = typyPRISM.closure.PercusYevick()
sys.omega[sys.types,sys.types] = typyPRISM.omega.DiscreteKoyama(epsilon=eps_bend,length=N,sigma=sigma,l=l)
sys.omega['A','B'] = typyPRISM.omega.InterMolecular()

lam_list = []
guess = np.zeros(sys.domain.length*sys.rank*sys.rank)
for i,epsilon in enumerate(np.arange(0.05,0.20,0.01)):
    epsilon *= -1.0
    print('T=',kT/abs(epsilon))
    print('1/T=',abs(epsilon)/kT)

    sys.potential['A','B'] = typyPRISM.potential.HardCoreLennardJones(epsilon=epsilon,sigma=sigma)

    PRISM = sys.createPRISM()
    PRISM.solve(guess,method='krylov')

    lam = typyPRISM.calculate.prism.spinodal_condition(PRISM)
    lam_list.append([abs(epsilon),lam['A','B']])
    
    guess = np.copy(PRISM.x)
    
    print(np.array(lam_list))

np.array(lam_list)

array([[  0.05      ,  10.03755618],
       [  0.06      ,   9.93573183],
       [  0.07      ,   9.8365841 ],
       [  0.08      ,   9.74023169],
       [  0.09      ,   9.64679469],
       [  0.1       ,   9.55639464],
       [  0.11      ,   9.46915468],
       [  0.12      ,   9.38519964],
       [  0.13      ,   9.30465622],
       [  0.14      ,   9.22765324],
       [  0.15      ,   9.15432181],
       [  0.16      ,   9.08479569],
       [  0.17      ,   9.01921167],
       [  0.18      ,   8.95770976],
       [  0.19      ,   8.9004343 ],
       [  0.2       ,   8.84753398]])

In [ ]:
%%opts Scatter (alpha=0.5)
%%opts Overlay [width=600,height=400]
%%output backend='matplotlib'

markers = {}
markers['A','A'] = 'o'
markers['A','B'] = '^'
markers['B','B'] = 'd'

curves = []
for i,(t1,t2),U in sys.potential.iterpairs():
    x = sys.domain.r
    y = U.calculate(x)/sys.kT
    miny = -1.1*abs(epsilon)
    maxy = +1.1*abs(epsilon)
    label = '{}-{}'.format(t1,t2)
    style = {}# {'marker':markers[t1,t2],'marker_size':12}
    c1 = hv.Curve((x,y),extents=(0,miny,4,maxy),label=label)(style=style)
    curves.append(c1)

hv.Overlay(curves)
    


In [ ]:
x= sys.domain.r
gr = typyPRISM.calculate.prism.pair_correlation(PRISM)

curves = []
for i,(t1,t2),y in gr.itercurve():
    label = '{}-{}'.format(t1,t2)
    c1 = hv.Curve((x,y),extents=(0,None,10,None),label=label)
    curves.append(c1)

hv.Overlay(curves)
    

In [ ]:
import typyPRISM
import scipy
import numpy as np

def density_correction(sigma,N,domain,omega_real):
    r = domain.r
    
    integrand  =(np.pi/6.0)*(sigma**(3.0)) 
    integrand *=(1 - 3*r/(2*sigma) + 0.5*(r/sigma)**(3.0))
    integrand *= omega_real
    integrand *= 4*np.pi*r*r
    
    integrand[r>sigma] = 0.0 # we only integrate inside the hard core
    delta  = scipy.integrate.simps(integrand,x=r)
    delta *= 1.0/(N*np.pi*sigma*sigma*sigma/6.0)
    
    factor = 1/(1-delta)
    return delta,factor

eps_bend = 0.0001
N = 20
l = sigma = 1.0
O = typyPRISM.omega.DiscreteKoyama(epsilon=eps_bend,length=N,sigma=sigma,l=l)
rho = sys.siteDensityMatrix[0,0]
omega_fourier = O.calculate2(sys.domain.k) 
omega_real = sys.domain.to_real(omega_fourier)
print(O.lp,np.arccos(O.cos0),np.pi/3)

density_correction(sigma,N,sys.domain,omega_real)

In [ ]:
0.099388516874491392*2.0

In [ ]:
from scipy.optimize import root
sigma = l = 1.0
lp = 1.0*l
# lp = 3*l
cos0    = 1 - sigma*sigma/(2.0 * l * l)
cos1 = l/lp - 1

def cos_avg(e,cos0):
    return 1/e  - ( np.exp(e) + cos0*np.exp(-e*cos0) )/( np.exp(e) - np.exp(-e*cos0) )

def funk(e):
    return cos_avg(e,cos0) - cos1



result = root(funk,0.1)
print(result)
print(cos_avg(result.x,cos0))










In [ ]:
epsilon = result.x
theta = np.arange(0,2*np.pi,0.01)
E = epsilon*(1 + np.cos(theta))
hv.Curve((theta,E))